In [6]:
import cv2
import numpy as np
import time

If the code was taken from github, then run this block to get to correct directory

In [7]:
%cd ../
%cd weights

/home/jp_zuniga/Yolo
/home/jp_zuniga/Yolo/weights


In [8]:
net = cv2.dnn.readNet("yolov3-tiny-custom_best (4).weights", "yolov3-tiny-custom.cfg")
classes = []
with open("obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

In [9]:
layer_names = net.getLayerNames()

Depending on the version of CV and NumPy you might have to run this code differently. Run the code below and if it errors out run the other code.

In [19]:
output_layers = [layer_names[i-1] for i in net.getUnconnectedOutLayers()]

In [11]:
holder = net.getUnconnectedOutLayers()
output_layers = []
for i in holder:
    output_layers.append(layer_names[int(i-1)])

In [12]:
colors = np.random.uniform(0, 255, size=(len(classes), 3))

In [14]:
cap = cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)
font = cv2.FONT_HERSHEY_PLAIN
frame_id = 1
if not cap.isOpened():
    raise ValueError("Error: Video capture not initialized.")

In [15]:

while True:
    starting_time = time.time()
    _, frame = cap.read()
    height, width, channels = frame.shape
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)
    
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.0:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 30), font, 1, color, 2)  

    elapsed_time = time.time() - starting_time
    fps = frame_id / elapsed_time
    cv2.putText(frame, "FPS: " + str(round(fps, 2)), (10, 50), font, 4, (0, 0, 0), 3)
    cv2.imshow("Image", frame)
    key = cv2.waitKey(1)
    #frame_id = frame_id + 1
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

Unable to init server: Could not connect: Connection refused


error: OpenCV(4.5.3) /tmp/pip-wheel-ulre47vq/opencv-contrib-python_edf6b92c2aed48ceb1cae55c8d5542a6/opencv/modules/highgui/src/window_gtk.cpp:635: error: (-2:Unspecified error) Can't initialize GTK backend in function 'cvInitSystem'


In [24]:
cap.release()
cv2.destroyAllWindows()